In [23]:
import sys
sys.path.insert(0,'C:\\Users\\Jacob\\Documents\\GitHub\\lenskit_confidence')

In [24]:
from lenskit.metrics import predict
import pandas as pd
import matplotlib.pyplot as plt

from lenskit.batch_ca import MultiEval
from lenskit.algorithms_ca import user_knn_ca, Recommender
from lenskit.metrics.predict import rmse
from lenskit import datasets
from lenskit.datasets import MovieLens
from lenskit import crossfold as xf
from lenskit import topn, datasets, batch_ca # *not* batch 
from lenskit import topn, util #, metrics
from lenskit.crossfold import partition_users, SampleN, partition_rows

In [25]:
util.log_to_notebook()

[   INFO] lenskit.util.log log already initialized
[   INFO] lenskit.util.log notebook logging configured
[   INFO] lenskit.util.log notebook logging configured


In [6]:
data = 'ml1m'

In [7]:
#model = 'CIBCF'  ##### DO AGAIN WITH r0 = 5
model = 'CUBCF'

In [8]:
path = 'D:\\results_conf_aware_nbhd_KDD2021\\' + data + '\\confidence_aware\\' + model + '\\'

In [21]:
list_lengths = list((1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100))
new_r0 = 4.25

In [22]:
for list_length in list_lengths:

    print(list_length)
    runs = pd.read_csv(path + 'runs.csv')
    runs.set_index('RunId', inplace = True)
    #runs.head()

    recs = pd.read_parquet(path + 'recommendations.parquet')
    #recs.head()

    recs['prediction'] = (recs['prediction']*recs['var']) + 4.5
    #recs.head()

    recs['score'] = (recs['prediction'] - new_r0)/recs['var']
    #recs.head()

    recs['rank'] = recs.groupby(['user', 'num_nbhr'])['score'].rank("dense", ascending = False)
    recs['score'] = recs['prediction']

    recs = recs[['item', 'score', 'user','rank','RunId']]
    #recs.head()

    recs = recs[recs['rank'] <= list_length]

    truth = pd.read_csv(path + 'truth.csv')
    truth = truth[['user', 'item', 'rating']]
    truth.head()

    rla = topn.RecListAnalysis()
    rla.add_metric(topn.ndcg) # precision, recall, recip_rank, dcg, ndcg
    rla.add_metric(topn.precision)
    raw_ndcg = rla.compute(recs, truth)
    #raw_ndcg.head()

    ### FOR NEIGHBORHOOD-BASED METHODS ONLY ###
    ndcg = raw_ndcg.join(runs[['name', 'nnbrs']], on = 'RunId')
    #ndcg.head()

    ### FOR NEIGHBORHOOD-BASED METHODS ONLY ###
    ndcg.fillna(0).groupby(['name', 'nnbrs'])['ndcg','precision'].mean()

    ndcg.fillna(0).groupby(['name', 'nnbrs'])['ndcg','precision'].mean().to_csv(path + 'results' + str(eval('list_length')) + '.csv')

1
[   INFO] lenskit.topn analyzing 180544 recommendations (30200 truth rows)
[   INFO] lenskit.topn using rec key columns ['RunId', 'user']
[   INFO] lenskit.topn using truth key columns ['user']
[   INFO] lenskit.topn collecting truth data
[   INFO] lenskit.topn collecting metric results
[   INFO] lenskit.sharing.shm serialized <lenskit.topn._RLAJob object at 0x000002512F5D1BB0> to 1474914 pickle bytes with 12084 buffers of 8427136 bytes
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.topn measured 69379 lists in 2m25.54s


<ipython-input-22-b94633faa281>:40: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ndcg.fillna(0).groupby(['name', 'nnbrs'])['ndcg','precision'].mean()
<ipython-input-22-b94633faa281>:42: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ndcg.fillna(0).groupby(['name', 'nnbrs'])['ndcg','precision'].mean().to_csv(path + 'results' + str(eval('list_length')) + '.csv')


2
[   INFO] lenskit.topn analyzing 353442 recommendations (30200 truth rows)
[   INFO] lenskit.topn using rec key columns ['RunId', 'user']
[   INFO] lenskit.topn using truth key columns ['user']
[   INFO] lenskit.topn collecting truth data
[   INFO] lenskit.topn collecting metric results
[   INFO] lenskit.sharing.shm serialized <lenskit.topn._RLAJob object at 0x000002512D203100> to 1474914 pickle bytes with 12084 buffers of 16034648 bytes
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.topn measured 70463 lists in 2m24.16s
3
[   INFO] lenskit.topn analyzing 517734 recommendations (30200 truth rows)
[   INFO] lenskit.topn using rec key columns ['RunId', 'user']
[   INFO] lenskit.topn using truth key columns ['user']
[   INFO] lenskit.topn collecting truth data
[   INFO] lenskit.topn collecting metric results
[   INFO] lenskit.sharing.shm serialized <lenskit.topn._RLAJob object at 0x000002512F4A4130> to 1474914 pickle bytes with 12084 buffer

[   INFO] lenskit.topn using truth key columns ['user']
[   INFO] lenskit.topn collecting truth data
[   INFO] lenskit.topn collecting metric results
[   INFO] lenskit.sharing.shm serialized <lenskit.topn._RLAJob object at 0x00000251327C1A90> to 1474914 pickle bytes with 12084 buffers of 213349216 bytes
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.topn measured 72480 lists in 2m9.17s
80
[   INFO] lenskit.topn analyzing 5060732 recommendations (30200 truth rows)
[   INFO] lenskit.topn using rec key columns ['RunId', 'user']
[   INFO] lenskit.topn using truth key columns ['user']
[   INFO] lenskit.topn collecting truth data
[   INFO] lenskit.topn collecting metric results
[   INFO] lenskit.sharing.shm serialized <lenskit.topn._RLAJob object at 0x000002512F415160> to 1474914 pickle bytes with 12084 buffers of 223155408 bytes
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.topn measured 72480 lis